In [6]:
from brian2 import *
import numpy as np
import logging
import warnings

from brian2 import prefs, set_device

# Tell Brian2 to use the Cython code generator:
prefs.codegen.target = 'cython'

Urd Here frist example: 

In [ ]:
# mini example 2x2 - ask from audience: 

from brian2 import *
import numpy as np
import warnings
import logging

# Optional: compile device (uncomment if you want runtime compilation)
# set_device('runtime')

# suppress warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
numpy.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)

# ----------------------------------------------------------------------------
# Spike timing and derivative (numpy implementations)

start_scope()
defaultclock.dt = 0.001*ms

@implementation('numpy', discard_units=True)
@check_units(w=1, time=1, result=1)
def spike_timing(w, time):
    # time is treated in ms (we pass raw numbers like 0.9, 1.2 ...)
    x = (time % 1.0)
    z = 10.0 * (x - 0.5)
    sigmoid_val = 1.0 / (1.0 + np.exp(-w * z))
    return sigmoid_val

@implementation('numpy', discard_units=True)
@check_units(w=1, time=1, result=1)
def d_spike_timing_dw(w, time):
    x = (time % 1.0)
    z = 5.0 * (x - 0.5)
    sig = 1.0 / (1.0 + np.exp(-w * z))
    return sig * (1.0 - sig) * z

# ----------------------------------------------------------------------------
# Forward pass for an (n_in)->(n_hidden)->(n_out) layer using Brian2

def layer_forward(inputs, W, layer_idx):
    """
    inputs: array of spike times (ms) shape (n_in,)
    W: weight matrix with bias row: shape (n_in+1, n_out)
    layer_idx: integer used inside synapse to tag layer (affects scheduled_time formula)
    returns: output spike times array shape (n_out,)
    """
    bias_time = 0.0
    aug_inputs = np.concatenate((inputs, [bias_time]))  # (n_in+1,)

    n_in_plus_bias, n_out = W.shape
    assert aug_inputs.size == n_in_plus_bias

    out_times = []
    # create a single postsynaptic neuron per output unit (run serially for clarity)
    for j in range(n_out):
        start_scope()
        defaultclock.dt = 0.001*ms

        G = NeuronGroup(1, '''
            v : 1
            sum : 1
            sr : 1
            scheduled_time : second
            global_clock : 1
            spiked : boolean
        ''', threshold='v>1', reset='''
            v = 0
            spiked = True
        ''', method='exact')

        # init
        G.v = G.sum = G.sr = 0
        G.global_clock = 0
        G.scheduled_time = 1e9*second

        stim = SpikeGeneratorGroup(n_in_plus_bias,
                                   indices=list(range(n_in_plus_bias)),
                                   times=aug_inputs*ms)

        S = Synapses(stim, G, '''w:1
            layer:1''',
            on_pre='''
            sr += 1
            sum += spike_timing(w, global_clock)
            scheduled_time = (sum/sr + layer + 0.004)*ms
        ''')
        S.connect(True)
        S.w = W[:, j]
        S.layer = layer_idx

        G.run_regularly('''
            v = (1.0 - spiked) * int(abs(t - scheduled_time) < 0.005*ms) * 1.2
            global_clock += 0.001
        ''', dt=0.001*ms)

        mon = SpikeMonitor(G)
        run(5*ms)

        ts = mon.spike_trains()[0]
        t0 = float(ts[0]/ms) if len(ts) > 0 else float(5.0)
        out_times.append(t0)

    return np.array(out_times)

# ----------------------------------------------------------------------------
# Training loop (2 -> 2 -> 2)

def train_snn_backprop(
    X, Y,
    W1_init, W2_init,
    epochs=10, lr=0.1,
    max_grad=50.0, w_min=-50.0, w_max=50.0,
    non_target_time=2.0,
    lam=0.5
):
    # W1 shape: (n_in+1, n_hidden), W2 shape: (n_hidden+1, n_out)
    W1 = W1_init.copy()
    W2 = W2_init.copy()

    beta = 0.9
    vW1 = np.zeros_like(W1)
    vW2 = np.zeros_like(W2)

    layer1_idx, layer2_idx = 1, 2
    N = len(X)

    for ep in range(epochs):
        acc_dW1 = np.zeros_like(W1)
        acc_dW2 = np.zeros_like(W2)
        epoch_loss = 0.0

        for xi, yi in zip(X, Y):
            h_times = layer_forward(xi, W1, layer1_idx)
            o_times = layer_forward(h_times, W2, layer2_idx)

            target_idx = np.argmax(yi)
            L_target = 0.5 * (o_times[target_idx] - yi[target_idx])**2
            non_ids = [j for j in range(len(o_times)) if j != target_idx]
            L_non = 0.5 * lam * sum([(o_times[j] - non_target_time)**2 for j in non_ids])
            L = L_target + L_non
            epoch_loss += L

            # output layer gradients
            delta_o = np.zeros_like(o_times)
            delta_o[target_idx] = (o_times[target_idx] - yi[target_idx])
            for j in non_ids:
                delta_o[j] = lam * (o_times[j] - non_target_time)

            aug_h = np.concatenate((h_times, [0.0]))
            dW2 = np.zeros_like(W2)
            for k in range(W2.shape[0]):
                for j in range(W2.shape[1]):
                    dW2[k, j] = delta_o[j] * d_spike_timing_dw(W2[k, j], aug_h[k])

            # backprop to hidden (fixed rule as in your code)
            delta_h = np.zeros_like(h_times)
            for k in range(len(h_times)):
                for j in range(W2.shape[1]):
                    dt_dw_output = d_spike_timing_dw(W2[k, j], aug_h[k])
                    delta_h[k] += delta_o[j] * W2[k, j] * dt_dw_output

            aug_xi = np.concatenate((xi, [0.0]))
            dW1 = np.zeros_like(W1)
            for i in range(W1.shape[0]):
                for k in range(W1.shape[1]):
                    dW1[i, k] = delta_h[k] * d_spike_timing_dw(W1[i, k], aug_xi[i])

            acc_dW1 += dW1
            acc_dW2 += dW2

        acc_dW1 /= N
        acc_dW2 /= N

        if ep % 5 == 0:
            print(f"  Gradient norms: ‖∇W1‖={np.linalg.norm(acc_dW1):.6f}, ‖∇W2‖={np.linalg.norm(acc_dW2):.6f}")

        lr1 = 2.0 * lr
        lr2 = lr

        g1_norm = np.linalg.norm(acc_dW1)
        g2_norm = np.linalg.norm(acc_dW2)
        if g1_norm > max_grad:
            acc_dW1 = acc_dW1 * (max_grad / g1_norm)
        if g2_norm > max_grad:
            acc_dW2 = acc_dW2 * (max_grad / g2_norm)

        vW1 = beta * vW1 + (1 - beta) * acc_dW1
        vW2 = beta * vW2 + (1 - beta) * acc_dW2

        W1 = np.clip(W1 - lr1 * vW1, w_min, w_max)
        W2 = np.clip(W2 - lr2 * vW2, w_min, w_max)

        print(f"Epoch {ep+1}/{epochs} — avg loss={epoch_loss/N:.4f}")
        print(f"             ‖W1‖={np.linalg.norm(W1):.3f}, ‖W2‖={np.linalg.norm(W2):.3f}")

    return W1, W2

# ----------------------------------------------------------------------------
# Example run
if __name__ == '__main__':
    np.random.seed(42)

    # Two input patterns (2 features)
    x0 = np.array([0.9, 0.4])
    x1 = np.array([0.8, 0.7])
    X = [x0 if i % 2 == 0 else x1 for i in range(8)]

    # Two target classes (2 outputs) -> target spike times (ms)
    y0 = np.array([2.0, 3.0])  # class 0: output0 early, output1 late
    y1 = np.array([3.0, 2.0])  # class 1: output1 early, output0 late
    Y = [y0 if i % 2 == 0 else y1 for i in range(8)]

    # initialize small weights (including bias rows)
    W1_0 = np.random.randn(3, 2) * 0.3  # (2 inputs + bias, 2 hidden)
    W2_0 = np.random.randn(3, 2) * 0.3  # (2 hidden + bias, 2 outputs)

    W1_tr, W2_tr = train_snn_backprop(X, Y, W1_0, W2_0, epochs=5, lr=20)

    # print('\nTrained W1:\n', W1_tr)
    # print('\nTrained W2:\n', W2_tr)

    # # quick inspection on the two prototype inputs
    # print('\nHidden times for x0:', layer_forward(x0, W1_tr, 1))
    # print('Hidden times for x1:', layer_forward(x1, W1_tr, 1))

    print('\n=== Test predictions ===')
    for xi, yi in zip(X, Y):
        h_times = layer_forward(xi, W1_tr, 1)
        o_times = layer_forward(h_times, W2_tr, 2)
        pred_class = np.argmax(o_times)
        true_class = np.argmax(yi)
        print(f"Input: {xi} -> outputs: {o_times} -> pred: {pred_class}, true: {true_class}")


  Gradient norms: ‖∇W1‖=0.007561, ‖∇W2‖=0.069418
Epoch 1/5 — avg loss=0.1765
             ‖W1‖=0.532, ‖W2‖=0.650
Epoch 2/5 — avg loss=0.1710
             ‖W1‖=0.543, ‖W2‖=0.769
Epoch 3/5 — avg loss=0.1656
             ‖W1‖=0.563, ‖W2‖=0.943
Epoch 4/5 — avg loss=0.1611
             ‖W1‖=0.590, ‖W2‖=1.147
Epoch 5/5 — avg loss=0.1561
             ‖W1‖=0.620, ‖W2‖=1.370

Trained W1:
 [[ 0.20517172 -0.07842559]
 [ 0.33105983  0.42822562]
 [-0.20687538 -0.00718386]]

Trained W2:
 [[ 0.73970015  0.10936166]
 [ 0.50994731 -0.44686367]
 [-0.54062519 -0.75242338]]

Hidden times for x0: [1.616 1.442]
Hidden times for x1: [1.682 1.551]

=== Test predictions ===
Input: [0.9 0.4] -> outputs: [2.689 2.69 ] -> pred: 1, true: 1
Input: [0.8 0.7] -> outputs: [2.765 2.656] -> pred: 0, true: 0
Input: [0.9 0.4] -> outputs: [2.689 2.69 ] -> pred: 1, true: 1
Input: [0.8 0.7] -> outputs: [2.765 2.656] -> pred: 0, true: 0
Input: [0.9 0.4] -> outputs: [2.689 2.69 ] -> pred: 1, true: 1
Input: [0.8 0.7] -> outputs

Next 2nd example: 

In [50]:
"""
Timing-based SNN-like MLP (NumPy) + backprop for classification (Iris).

This version:
- Avoids data leakage (scales only train set).
- Reports untrained accuracy before training.
- Adds training stabilizers: LR warmup, margin ramp, gradient clipping.
"""

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# ------------------------- Spike timing function + derivatives -------------------------
alpha = 10.0   # shape factor in z = alpha*(x-0.5)

def spike_timing(w, t):
    x = np.mod(t, 1.0)
    z = alpha * (x - 0.5)
    sig = 1.0 / (1.0 + np.exp(-w * z))
    return sig

def d_spike_timing_dw(w, t):
    x = np.mod(t, 1.0)
    z = alpha * (x - 0.5)
    sig = 1.0 / (1.0 + np.exp(-w * z))
    return sig * (1.0 - sig) * z

def d_spike_timing_dt(w, t):
    x = np.mod(t, 1.0)
    z = alpha * (x - 0.5)
    sig = 1.0 / (1.0 + np.exp(-w * z))
    return sig * (1.0 - sig) * w * alpha

# ------------------------- Utilities -------------------------

def softmax(logits, temp=1.0):
    l = (logits - np.max(logits)) / temp
    ex = np.exp(l)
    return ex / np.sum(ex)

def one_hot(y, n_classes):
    v = np.zeros(n_classes)
    v[y] = 1.0
    return v

def features_to_spiketimes_minmax(x_raw, mins, maxs, tmin=0.05, tmax=1.0):
    denom = (maxs - mins).copy()
    denom[denom == 0] = 1.0
    norm = (x_raw - mins) / denom  # in [0,1]
    times = tmin + (1.0 - norm) * (tmax - tmin)
    return times

# ------------------------- Forward pass (timing simulation style) -------------------------

def forward_timing(sample_times, W1, W2, layer_offsets=(0.06, 0.14)):
    n_in = sample_times.shape[0]
    n_hidden = W1.shape[1]
    n_out = W2.shape[1]

    # input -> hidden
    aug_in_times = np.concatenate([sample_times, np.array([0.0])])  # bias spike at 0
    sr_hidden = float(aug_in_times.size)
    hidden_times = np.zeros(n_hidden)

    for k in range(n_hidden):
        s = 0.0
        for i, t_i in enumerate(aug_in_times):
            st = spike_timing(W1[i, k], t_i)
            s += st
        hidden_times[k] = s / sr_hidden + layer_offsets[0]

    # hidden -> output
    aug_hidden = np.concatenate([hidden_times, np.array([0.0])])
    sr_out = float(aug_hidden.size)
    out_times = np.zeros(n_out)

    for j in range(n_out):
        s = 0.0
        for k, t_k in enumerate(aug_hidden):
            st = spike_timing(W2[k, j], t_k)
            s += st
        out_times[j] = s / sr_out + layer_offsets[1]

    info = {
        'aug_in_times': aug_in_times, 'sr_hidden': sr_hidden,
        'hidden_times': hidden_times,
        'aug_hidden': aug_hidden, 'sr_out': sr_out,
        'out_times': out_times
    }
    return out_times, info

# ------------------------- Backpropagation through timing -------------------------

def grads_timing_given_dL_dlogits(info, W1, W2, dL_dlogits):
    aug_in = info['aug_in_times']
    sr_h = info['sr_hidden']
    hidden_times = info['hidden_times']
    aug_h = info['aug_hidden']
    sr_o = info['sr_out']
    out_times = info['out_times']

    n_in_plus_bias, n_hidden = W1.shape
    n_hidden_plus_bias, n_out = W2.shape

    dW2 = np.zeros_like(W2)
    dW1 = np.zeros_like(W1)

    # W2 grads
    for j in range(n_out):
        for k in range(n_hidden_plus_bias):
            t_k = aug_h[k]
            w = W2[k, j]
            d_st_dw = d_spike_timing_dw(w, t_k)
            dL_dw = dL_dlogits[j] * (1.0 / sr_o) * d_st_dw * (-1.0)  # logit = -t
            dW2[k, j] = dL_dw

    # W1 grads: chain through hidden -> outputs
    for i in range(n_in_plus_bias):
        t_i = aug_in[i]
        for k in range(n_hidden):
            chain = 0.0
            t_hidden_k = hidden_times[k]
            for j in range(n_out):
                w2_kj = W2[k, j]
                d_st_dt = d_spike_timing_dt(w2_kj, t_hidden_k)
                dtout_dhk = (1.0 / sr_o) * d_st_dt
                chain += dL_dlogits[j] * dtout_dhk * (-1.0)
            d_sum_dw1 = d_spike_timing_dw(W1[i, k], t_i)
            dW1[i, k] = chain * (1.0 / sr_h) * d_sum_dw1

    return dW1, dW2

# ------------------------- Metrics & Helpers -------------------------

def predict_times_to_class(out_times, temp=0.5):
    probs = softmax(-out_times, temp=temp)
    return int(np.argmax(probs)), probs

def confusion_matrix(preds, truths, n_classes=3):
    cm = np.zeros((n_classes, n_classes), dtype=int)
    for p, t in zip(preds, truths):
        cm[t, p] += 1
    return cm

# ------------------------- Stabilizers -------------------------

def clip_grad_(g, clip=1.0):
    np.clip(g, -clip, clip, out=g)

def linear_warmup(epoch, warmup_epochs, base_lr):
    if warmup_epochs <= 0:
        return base_lr
    return base_lr * min(1.0, epoch / warmup_epochs)

def ramp_lambda(epoch, ramp_epochs, target_lambda):
    if ramp_epochs <= 0:
        return target_lambda
    return target_lambda * min(1.0, epoch / ramp_epochs)

# ------------------------- Training with Adam + margin loss -------------------------

def train_iris_timing_adam(hidden_size=12, lr=0.02, epochs=100, temp=0.60, print_every=5,
                            lam_margin=0.6, beta_margin=4.0, weight_decay=1e-5,
                            warmup_epochs=0, lambda_ramp_epochs=8, grad_clip=1.0):
    iris = load_iris()
    X_raw = iris.data.copy()
    y = iris.target.copy()
    n_classes = 3

    # Split, then scale train/test separately
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X_raw, y, test_size=0.25, random_state=14, stratify=y)

    mm = MinMaxScaler()
    X_train_mm = mm.fit_transform(X_train_raw)
    X_test_mm  = mm.transform(X_test_raw)

    mins = np.zeros(X_raw.shape[1]); maxs = np.ones(X_raw.shape[1])
    X_train_times = np.array([features_to_spiketimes_minmax(s, mins, maxs, 0.05, 1.0) for s in X_train_mm])
    X_test_times  = np.array([features_to_spiketimes_minmax(s, mins, maxs, 0.05, 1.0) for s in X_test_mm])
    X_full_mm     = mm.transform(X_raw)
    X_full_times  = np.array([features_to_spiketimes_minmax(s, mins, maxs, 0.05, 1.0) for s in X_full_mm])

    n_in = X_train_times.shape[1]
    n_hidden = hidden_size
    n_out = n_classes

    rng = np.random.RandomState(14)
    W1 = rng.normal(scale=0.12, size=(n_in + 1, n_hidden))
    W2 = rng.normal(scale=0.12, size=(n_hidden + 1, n_out))

    # Adam state
    mW1 = np.zeros_like(W1); vW1 = np.zeros_like(W1)
    mW2 = np.zeros_like(W2); vW2 = np.zeros_like(W2)
    beta1, beta2 = 0.9, 0.999
    eps = 1e-8
    tstep = 0

    layer_offsets = (0.06, 0.14)

    def predict_set_times(Xset_times, temp_local=temp):
        preds = []
        for x_t in Xset_times:
            out_times, _ = forward_timing(x_t, W1, W2, layer_offsets)
            pred, _ = predict_times_to_class(out_times, temp=temp_local)
            preds.append(pred)
        return np.array(preds)

    # --- initial (untrained) accuracy ---
    print(f"Initial (untrained) — train_acc={np.mean(predict_set_times(X_train_times)==y_train):.3f} "
          f"test_acc={np.mean(predict_set_times(X_test_times)==y_test):.3f}")

    for epoch in range(1, epochs + 1):
        lr_now = linear_warmup(epoch, warmup_epochs, lr)
        lam_now = ramp_lambda(epoch, lambda_ramp_epochs, lam_margin)

        idx = np.arange(len(X_train_times))
        rng.shuffle(idx)
        total_loss = 0.0

        for ii in idx:
            x_t = X_train_times[ii]
            y_true = int(y_train[ii])

            out_times, info = forward_timing(x_t, W1, W2, layer_offsets)
            logits = -out_times
            probs = softmax(logits, temp=temp)

            target = one_hot(y_true, n_out)

            # CE loss
            loss_ce = -np.sum(target * np.log(probs + 1e-12))

            # Margin loss (with ramp)
            mask = np.ones_like(out_times, dtype=bool); mask[y_true] = False
            exp_terms = np.exp(-beta_margin * (out_times[mask] - out_times[y_true]))
            loss_margin = lam_now * np.sum(exp_terms)

            # dL/dt for margin
            dLdt = np.zeros_like(out_times)
            dLdt[mask] = -lam_now * beta_margin * np.exp(-beta_margin * (out_times[mask] - out_times[y_true]))
            dLdt[y_true] = lam_now * beta_margin * np.sum(np.exp(-beta_margin * (out_times[mask] - out_times[y_true])))

            # combine grads (logit = -t)
            dL_dlogits = (probs - target) + (-dLdt)

            dW1, dW2 = grads_timing_given_dL_dlogits(info, W1, W2, dL_dlogits)

            # weight decay
            if weight_decay:
                dW1 += weight_decay * W1
                dW2 += weight_decay * W2

            # clip
            clip_grad_(dW1, grad_clip)
            clip_grad_(dW2, grad_clip)

            # Adam update
            tstep += 1
            mW1 = beta1 * mW1 + (1 - beta1) * dW1
            vW1 = beta2 * vW1 + (1 - beta2) * (dW1 * dW1)
            mW1_hat = mW1 / (1 - beta1 ** tstep)
            vW1_hat = vW1 / (1 - beta2 ** tstep)
            W1 -= lr_now * mW1_hat / (np.sqrt(vW1_hat) + eps)

            mW2 = beta1 * mW2 + (1 - beta1) * dW2
            vW2 = beta2 * vW2 + (1 - beta2) * (dW2 * dW2)
            mW2_hat = mW2 / (1 - beta1 ** tstep)
            vW2_hat = vW2 / (1 - beta2 ** tstep)
            W2 -= lr_now * mW2_hat / (np.sqrt(vW2_hat) + eps)

            total_loss += (loss_ce + loss_margin)

        if epoch % print_every == 0 or epoch == 1:
            train_acc = np.mean(predict_set_times(X_train_times)==y_train)
            test_acc  = np.mean(predict_set_times(X_test_times)==y_test)
            print(f"Epoch {epoch:3d}"
                  f" loss={total_loss/len(X_train_times):.4f} acc_test={test_acc:.3f} acc_train={test_acc:.3f}")

    # final evaluation
    preds_all = predict_set_times(X_full_times)
    full_acc = np.mean(preds_all == y)
    print(f"\nFinal accuracy on full Iris dataset (timing SNN + Adam + margin): {full_acc:.3f}")

    preds_test = predict_set_times(X_test_times)
    cm = confusion_matrix(preds_test, y_test, n_classes=n_out)
    print("\nConfusion matrix (rows=true, cols=pred):")
    print(cm)

    return W1, W2

import random
random.seed(42)
if __name__ == '__main__':
    W1, W2 = train_iris_timing_adam(hidden_size=50, lr=0.001, epochs=100, print_every=20)


Initial (untrained) — train_acc=0.330 test_acc=0.342
Epoch   1 loss=1.2434 acc_test=0.579 acc_train=0.579
Epoch  20 loss=1.4685 acc_test=0.974 acc_train=0.974
Epoch  40 loss=1.3563 acc_test=0.947 acc_train=0.947
Epoch  60 loss=1.2737 acc_test=0.921 acc_train=0.921
Epoch  80 loss=1.2014 acc_test=0.895 acc_train=0.895
Epoch 100 loss=1.1423 acc_test=0.895 acc_train=0.895

Final accuracy on full Iris dataset (timing SNN + Adam + margin): 0.927

Confusion matrix (rows=true, cols=pred):
[[12  1  0]
 [ 1 11  0]
 [ 0  2 11]]


Demo 3 Ymir

In [18]:
# v5 Ymir gets X-Y-Z so that 3 layers: 

# v4 : Ymir with varying inputs and outputs x-(x*y)-y network

from brian2 import *
import numpy as np
import logging, warnings

start_scope()

defaultclock.dt = 0.01*ms  

#prefs.codegen.target = 'cython'
set_device('runtime')
warnings.filterwarnings('ignore', category=RuntimeWarning)  # check error later
np.seterr(over='ignore', under='ignore')
logging.getLogger('brian2').setLevel(logging.ERROR)


decay_rate = 5*ms

def run_Ymir(inputs, y, z, taus1, taus2):

    x = len(inputs)
    xy_ind = x*y
    yz_ind = y*z


    if len(taus1) != xy_ind:
        print("not the right size for taus1 buddy")
        raise ValueError(f"Length of taus1 wrong .")
    
    if len(taus2) != yz_ind:
        print("not the right size for taus2 buddy")
        raise ValueError(f"Length of taus2 is wrong")

    input_neurons = NeuronGroup(x,'''
    dv/dt = -v/ decay_rate : volt                 
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    input_neurons.v = 0 * volt

    indices_input = []
    for i in range(0, x):
        indices_input.append(i)

    stim_input = SpikeGeneratorGroup(x, indices=indices_input, times= inputs * ms)

    syn_input = Synapses(stim_input, input_neurons[0:x], '''
    ''', on_pre='''
        v += 1.2 * volt
    ''')
    syn_input.connect(j='i') 

    ind_input_neurons = NeuronGroup(xy_ind,
    '''dv/dt = -v/ decay_rate : volt 
         
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    ind_input_neurons.v = 0 * volt

    input_range = []
    for i in range(0, x):
        for j in range(0, y):
            input_range.append(i)


    ind_input = Synapses(input_neurons[0:x], ind_input_neurons[0:xy_ind], '''
    ''', on_pre='''
        v += 0.55 * volt
    ''')
    ind_input.connect(i=input_range, j=[k for k in range(0, xy_ind)]) # look into what j is later


    stim_tau_hidden = SpikeGeneratorGroup(xy_ind, indices=[k for k in range(0, xy_ind)], times = taus1 * ms)

    syn_tau_hidden = Synapses(stim_tau_hidden, ind_input_neurons[0:xy_ind], '''
    ''', on_pre='''
        v += 0.55 * volt
    ''')
    syn_tau_hidden.connect(j='i')

    hidden_neurons = NeuronGroup(y,
    '''dv/dt = -v/ decay_rate : volt 
         
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    hidden_neurons.v = 0 * volt

    output_range = []
    for i in range(0, x):
        for j in range(0, y):
            output_range.append(j)

    syn_ind_hidden = Synapses(ind_input_neurons[0:xy_ind], hidden_neurons[0:y], '''
    ''', on_pre='''
        v += 1.2 * volt
    ''')

    syn_ind_hidden.connect(i=[k for k in range(0, xy_ind)], j=output_range)

    # last layer copy of above below but changed names same logic -- can functionalize later

    # stim_input = SpikeGeneratorGroup(x, indices=indices_input, times= inputs * ms)

    # syn_input = Synapses(stim_input, input_neurons[0:x], '''
    # ''', on_pre='''
    #     v += 1.2 * volt
    # ''')
    # syn_input.connect(j='i') 

    ind_hidden_neurons = NeuronGroup(yz_ind,
    '''dv/dt = -v/ decay_rate : volt 
         
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    ind_hidden_neurons.v = 0 * volt

    hidden_range = []
    for i in range(0, y):
        for j in range(0, z):
            hidden_range.append(i)


    syn_hidden_output = Synapses(hidden_neurons[0:y], ind_hidden_neurons[0:yz_ind], '''
    ''', on_pre='''
        v += 0.55 * volt
    ''')
    syn_hidden_output.connect(i=hidden_range, j=[k for k in range(0, yz_ind)])


    stim_tau_output = SpikeGeneratorGroup(yz_ind, indices=[k for k in range(0, yz_ind)], times = taus2 * ms)

    syn_tau_output = Synapses(stim_tau_output, ind_hidden_neurons[0:yz_ind], '''
    ''', on_pre='''
        v += 0.55 * volt
    ''')
    syn_tau_output.connect(j='i')

    output_neurons = NeuronGroup(z,
    '''dv/dt = -v/ decay_rate : volt 
         
        ''',
    threshold='v > 1.0 * volt',
    reset='v = 0 * volt',
    method='exact')
    output_neurons.v = 0 * volt

    output_out_range = []
    for i in range(0, y):
        for j in range(0, z):
            output_out_range.append(j)

    syn_output = Synapses(ind_hidden_neurons[0:yz_ind], output_neurons[0:z], '''
    ''', on_pre='''
        v += 1.2 * volt
    ''')

    syn_output.connect(i=[k for k in range(0, yz_ind)], j=output_out_range)

    

    mon = StateMonitor(input_neurons, 'v', record=True, dt=0.01*ms)
    M1 = StateMonitor(ind_input_neurons, 'v', record=True, dt=0.01*ms)

    M2 = StateMonitor(hidden_neurons, 'v', record=True, dt=0.01*ms)

    M3 = StateMonitor(ind_hidden_neurons, 'v', record=True, dt=0.01*ms)
    M4 = StateMonitor(output_neurons, 'v', record=True, dt=0.01*ms)

    spikemon = SpikeMonitor(input_neurons)
    spikemon_1 = SpikeMonitor(ind_input_neurons)
    spikemon_2 = SpikeMonitor(hidden_neurons)
    spikemon_3 = SpikeMonitor(ind_hidden_neurons)
    spikemon_4 = SpikeMonitor(output_neurons)

    run(10*ms)

    # # Plot v
    # figure(figsize=(10, 6))
    # for i in range(0, yz_ind): 
    #     #plot(mon.t/ms, mon.v[i], label=f'Neuron {i}')
    #     plot(M2.t/ms, M2.v[i], label=f'Neuron {i}')
    #     plot(M4.t/ms, M4.v[i], label=f'output neruon')
    # # for i in range(0, z): 
    # #     plot(M4.t/ms, M2.v[i], label=f'Neuron output')
    # xlabel('Time (ms)')
    # ylabel('Membrane potential')
    # legend()
    # title('SNN Spike Propagation Across All Layers')
    # show()

    results = []
    results.append(spikemon.spike_trains())
    results.append(spikemon_1.spike_trains())
    results.append(spikemon_2.spike_trains())
    results.append(spikemon_3.spike_trains())
    results.append(spikemon_4.spike_trains())
    


    #print(results)
    return results   # check the dictionalry apect of this as the odder might be changing so harder wirinign i-j such relationship



inputs = [1.0]
y = 2
z = 2

h_neurons_sumed_movements = [0] * y 

taus1 = [1.1, 1.1] 
taus2 = [1.2, 1.3, 1.35, 1.4] 


# funcition for training taus
def see_system_run():
    raw_results = run_Ymir(inputs, y, z, taus1, taus2)

    recorded_spikes = [
        {neuron_idx: (spike_times / ms).tolist()  # convert to float ms
        for neuron_idx, spike_times in group.items()}
        for group in raw_results
    ]

    return(recorded_spikes)

    # for i in range(len(recorded_spikes)):
    #     print(f"Group {i} spikes:")
    #     for neuron_idx, spike_times in recorded_spikes[i].items():
    #         print(f"  Neuron {neuron_idx}: {spike_times}")
    #     print()



def tau_shifer(spike, tau, i, strength, closer=True):
    global h_neurons_sumed_movements
    # i is equal to what syn we are on 0 and 2 are n0 and 1 & 3 --> n1
    delta = spike - tau
    j = int(i > 1)

    if closer:
        # Original behavior: bigger delta → bigger step toward spike
        step = delta * strength
        if tau + step < 0:
            h_neurons_sumed_movements[j] += 0.02
            return 0.02
        h_neurons_sumed_movements[j] += step
        return tau + step
    else:
        # Inverse scaling: bigger delta → smaller step
        if delta == 0:
            h_neurons_sumed_movements[j] += 0.02
            return tau + 0.02  # struck will always fire if - is in range? - make sure 0 is always out of range for possilbe inputs? 
        step = (strength / abs(delta)) * delta  # sign from delta
        if tau - step < 0:
            h_neurons_sumed_movements[j] += 0.02
            return 0.02
        h_neurons_sumed_movements[j] -= step
        return tau - step
        
def directions(output_spikes, desired):
     # just firinng for a single spike or not change change later
    mods = [-1] * len(desired)  # assuming we want to modify all neurons
    
    for i in range(len(desired)): 
        if desired[i] == True:    # if we wanted a spike
            if output_spikes[i] == []: # if no spike saw
                mods[i] = 2 
            else:                   # if spike saw
                mods[i] = 0
        else:
            if output_spikes[i] == []: # if we didnt want a spike and it didnt spike
                mods[i] = 3
            else:   # if we didnt want it to spike but it did
                mods[i] = 1
    return mods     


def train_taus_last_layer(inputs, taus1, taus2, strength, desired):

    # run it and gets data with current values
    raw_results = run_Ymir(inputs, y, z, taus1, taus2)

    recorded_spikes = [
        {neuron_idx: (spike_times / ms).tolist()  # convert to float ms
        for neuron_idx, spike_times in group.items()}
        for group in raw_results
    ]
     # here we have the data of all the spikes and taus and desired we send into to get new taus
    
    d = directions(recorded_spikes[2], desired)  # get the directions for the taus
    
    #print(f"directions: {d}")

    # have function for sorting and making bools for such


    # basied on direction get proper assocaltion for each direction and strength for updating and send each though tau shifter
    move = False
    count = 0
    index = 0
    for i in range((len(taus2))):
        # if spike as well
            #if i % 2 == 0: # even index taus        # will need to change to extend to multi later
        if d[index] % 2 == 0:  # 0 and 2 here
            move = True
        else:
            move = False
        if recorded_spikes[2][count] != []:  # if we saw a spike
            # print("count:", count)
            # print(i)
            # print("taus ", taus2[i])
            # if recorded_spikes[3][count][0] != []:
            #     print(f"{i}")
            #     print("spike at: ", recorded_spikes[3][count][0])
            #     print("taus2: ", taus2[i])
            #     print("strength: ", strength)
            #     print("move: ", move)
            new_tau = tau_shifer(recorded_spikes[2][count][0], taus2[i], i, strength, closer=move)
            #print("new tay and old tau: ", new_tau, taus2[i])
            taus2[i] = new_tau
    
        else: 
            #new_tau = tau_shifer(0, taus2[i], strength, closer=move)  # if no spike saw then just use 0
            print("condition of change on condition of hidden not having spike ")# DID YOU EVER GOT HEREEEE")
            #print("no spike saw so no update?" )#new tay and old tau: ", new_tau, taus2[i])
            #taus2[i] = new_tau

        if (i+1) % 2 == 0:
            count += 1
        if index == 0:
            index += 1
        elif index == 1:
            index = 0
    

    return #recorded_spikes[3]  # return the last layer spikes for now

    # example 2 varying inputs to train specifics 1.0 goes to T/F and 1.5 goes to F/T 


In [19]:
y = 2
z = 2
taus1 = [1.1, 1.1] 
taus2 = [1.2, 1.3, 1.35, 1.4] 

see_system_run()

desired_1 = [True, False]
desired_2 = [False, True]
for i in range(20):
    train_taus_last_layer([1.0], taus1, taus2, .1, desired_1)
    train_taus_last_layer([1.5], taus1, taus2, .1, desired_2)

inputs = [1.0]
print("inputs 1 output_spikes" , see_system_run()[4], " desired: ", desired_1)
inputs = [1.5]
print("inputs 2 output_spikes" , see_system_run()[4],  " desired: ", desired_2)

inputs 1 output_spikes {0: [1.1400000000000001], 1: []}  desired:  [True, False]
inputs 2 output_spikes {0: [], 1: [2.31]}  desired:  [False, True]


In [1]:
# with larger model here 